In [5]:
pip install pandas mysql-connector-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import mysql.connector
from pathlib import Path
import os

# 1) CSV 읽기 (원래 utf8 파일 그대로 사용)
# 워크스페이스 루트 기준 상대 경로 사용
# 노트북 실행 위치가 scripts 폴더라면 상위 폴더로 이동
base_dir = Path(os.getcwd()).parent if Path(os.getcwd()).name == "scripts" else Path(os.getcwd())
csv_path = base_dir / "서울시_상권_카페빵_표본_with_transit_utf8.csv"

# 파일이 없으면 절대 경로로 시도
if not csv_path.exists():
    csv_path = Path(r"C:\Users\jiwon\Desktop\연구\space-agent\서울시_상권_카페빵_표본_with_transit.csv")

df = pd.read_csv(
    csv_path,
    encoding="utf-8"
)

print(f"파일 경로: {csv_path}")
print(df.head())  # 잘 읽히는지 한번 확인

파일 경로: c:\Users\jiwon\Desktop\연구\space-agent\서울시_상권_카페빵_표본_with_transit_utf8.csv
            cafe_name category district  dong_code dong_name  law_dong_code  \
0             가마쿠라도산점     빵/도넛      강남구   11680545      압구정동     1168010700   
1              강남주막다방       카페      강남구   11680510       신사동     1168010700   
2  고센뷰티&커스텀커피&더뷰도산도산점       카페      강남구   11680545      압구정동     1168010700   
3           공차SSG푸드마켓       카페      강남구   11680656      도곡2동     1168011800   
4           과일에반하다프루타       카페      강남구   11680590      삼성2동     1168010500   

  law_dong_name               address           road_address level  \
0           신사동  서울특별시 강남구 신사동 657-16   서울특별시 강남구 선릉로155길 28     1   
1           신사동   서울특별시 강남구 신사동 541-9      서울특별시 강남구 가로수길 16     1   
2           신사동   서울특별시 강남구 신사동 634-9  서울특별시 강남구 언주로168길 6-9   NaN   
3           도곡동  서울특별시 강남구 도곡동 467-17    서울특별시 강남구 언주로30길 57     1   
4           삼성동   서울특별시 강남구 삼성동 48-13   서울특별시 강남구 선릉로112길 87     1   

    longitude   latitud

In [8]:
# 2) RDS 접속
conn = mysql.connector.connect(
    host="space-db.c9kgsw8kchyu.ap-northeast-2.rds.amazonaws.com",
    user="jiwon",          # 네 RDS 유저명
    password="010lama!!",    # 네 비밀번호
    database="cafe",       # workbench에서 만든 schema 이름
)
cur = conn.cursor()

In [9]:
# 3) 테이블 만들기 (컬럼명은 네 CSV 헤더에 맞게 수정)
cur.execute("""
CREATE TABLE IF NOT EXISTS cafe_sample (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cafe_name        VARCHAR(255),
    category         VARCHAR(100),
    district         VARCHAR(50),
    dong_code        INT,
    dong_name        VARCHAR(100),
    law_dong_code    INT,
    law_dong_name    VARCHAR(100),
    address          VARCHAR(255),
    road_address     VARCHAR(255),
    level            VARCHAR(50),
    longitude        DOUBLE,
    latitude         DOUBLE,
    nearest_station  VARCHAR(100),
    transit_type     VARCHAR(50),
    walk_time_minutes INT,
    walk_distance_m   DOUBLE,
    straight_distance_m DOUBLE
) CHARACTER SET utf8mb4;
""")

In [14]:
# 4) INSERT 실행
cols = [
    "cafe_name", "category", "district", "dong_code", "dong_name",
    "law_dong_code", "law_dong_name", "address", "road_address",
    "level", "longitude", "latitude", "nearest_station",
    "transit_type", "walk_time_minutes", "walk_distance_m",
    "straight_distance_m"
]

# NaN 값을 None으로 변환 (MySQL의 NULL로 처리)
import numpy as np
df_insert = df[cols].copy()

# 정수형 컬럼 인덱스 찾기
int_cols = ["dong_code", "law_dong_code", "walk_time_minutes"]
int_col_indices = [i for i, col in enumerate(cols) if col in int_cols]

# 각 행을 순회하면서 numpy.nan을 None으로 명시적으로 변환
data_to_insert = []
for _, row in df_insert.iterrows():
    row_list = []
    for idx, val in enumerate(row):
        # numpy.nan, pandas.NA, None 모두 처리
        if pd.isna(val) or val is None:
            row_list.append(None)
        elif isinstance(val, float) and np.isnan(val):
            row_list.append(None)
        else:
            # 정수형 컬럼 처리
            if idx in int_col_indices:
                try:
                    row_list.append(int(val))
                except (ValueError, TypeError):
                    row_list.append(None)
            else:
                row_list.append(val)
    data_to_insert.append(row_list)

# 첫 번째 행 확인 (디버깅용)
if len(data_to_insert) > 0:
    print(f"첫 번째 행 샘플: {data_to_insert[0][:5]}...")
    print(f"첫 번째 행에 None 개수: {sum(1 for x in data_to_insert[0] if x is None)}")

placeholders = ",".join(["%s"] * len(cols))
sql = f"INSERT INTO cafe_sample ({','.join(cols)}) VALUES ({placeholders})"

cur.executemany(sql, data_to_insert)
conn.commit()

print(f"✅ CSV → MySQL 업로드 완료 (총 {len(data_to_insert)}개 행)")


첫 번째 행 샘플: ['가마쿠라도산점', '빵/도넛', '강남구', 11680545, '압구정동']...
첫 번째 행에 None 개수: 0
✅ CSV → MySQL 업로드 완료 (총 2500개 행)


In [9]:
# cafe_sample 테이블 중복 데이터 제거
# 각 (cafe_name, district, dong_name) 조합에 대해 id가 가장 작은 것만 남기고 나머지 삭제

import mysql.connector

conn = mysql.connector.connect(
    host="space-db.c9kgsw8kchyu.ap-northeast-2.rds.amazonaws.com",
    user="jiwon",
    password="010lama!!",
    database="cafe",
)
cur = conn.cursor()

# 현재 행 수 확인
cur.execute("SELECT COUNT(*) FROM cafe_sample")
current_count = cur.fetchone()[0]
print(f"현재 cafe_sample 행 수: {current_count}개")

# 중복 제거: 각 (cafe_name, district, dong_name) 조합에 대해 id가 가장 작은 것만 남기기
cur.execute("""
DELETE t1 FROM cafe_sample t1
INNER JOIN cafe_sample t2 
WHERE t1.id > t2.id 
  AND t1.cafe_name = t2.cafe_name 
  AND t1.district = t2.district 
  AND t1.dong_name = t2.dong_name;
""")

deleted_count = cur.rowcount
conn.commit()

# 삭제 후 행 수 확인
cur.execute("SELECT COUNT(*) FROM cafe_sample")
final_count = cur.fetchone()[0]

print(f"삭제된 중복 행 수: {deleted_count}개")
print(f"최종 cafe_sample 행 수: {final_count}개")

cur.close()
conn.close()


현재 cafe_sample 행 수: 5000개
삭제된 중복 행 수: 2503개
최종 cafe_sample 행 수: 2497개


In [ ]:
cur.close()
conn.close()

In [ ]:
# 1) 리뷰 CSV 로드
from pathlib import Path
import os

# 파일 경로 찾기
base_dir = Path(os.getcwd()).parent if Path(os.getcwd()).name == "scripts" else Path(os.getcwd())
csv_path = base_dir / "google_reviews_sample.csv"

if not csv_path.exists():
    csv_path = Path(r"C:\Users\jiwon\Desktop\연구\space-agent\google_reviews_sample.csv")

print(f"리뷰 CSV 경로: {csv_path}")

df = pd.read_csv(csv_path, encoding="utf-8")
print(f"리뷰 데이터 로드 완료: {len(df)}개 행")

# 컬럼명 정리 (영어로 바꾸기)
df.rename(columns={
    "상호명": "cafe_name",
    "시군구명": "district",
    "행정동명": "dong_name",
    "작성자": "reviewer_name",
    "평점": "rating",
    "리뷰": "review_text",
    "작성일": "review_date",
    "언어": "lang"
}, inplace=True)

# 2) MySQL 연결
conn = mysql.connector.connect(
    host="space-db.c9kgsw8kchyu.ap-northeast-2.rds.amazonaws.com",
    user="jiwon",          # 네 RDS 유저명
    password="010lama!!",    # 네 비밀번호
    database="cafe",       # workbench에서 만든 schema 이름
)
cur = conn.cursor()

# 2-1) cafe_review 테이블 생성
cur.execute("""
CREATE TABLE IF NOT EXISTS cafe_review (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cafe_id INT,
    cafe_name VARCHAR(255),
    district VARCHAR(50),
    dong_name VARCHAR(100),
    place_id VARCHAR(255),
    lat DOUBLE,
    lng DOUBLE,
    reviewer_name VARCHAR(255),
    rating INT,
    review_text TEXT,
    review_date VARCHAR(100),
    lang VARCHAR(10),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (cafe_id) REFERENCES cafe_sample(id) ON DELETE CASCADE,
    INDEX idx_cafe_id (cafe_id),
    INDEX idx_cafe_name (cafe_name),
    INDEX idx_district (district)
) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
""")
conn.commit()
print("✅ cafe_review 테이블 생성 완료")

# 3) cafe_sample 테이블 가져오기 (for matching)
cur.execute("SELECT id, cafe_name, district, dong_name FROM cafe_sample")
cafe_list = cur.fetchall()
cafe_df = pd.DataFrame(cafe_list, columns=["id", "cafe_name", "district", "dong_name"])
print(f"카페 샘플 데이터 로드 완료: {len(cafe_df)}개 카페")

# 4) 리뷰 CSV와 카페 목록 매칭
merged = df.merge(
    cafe_df,
    how="left",
    on=["cafe_name", "district", "dong_name"]
)

merged.rename(columns={"id": "cafe_id"}, inplace=True)

# 매칭 안 된 리뷰 확인
unmatched = merged[merged["cafe_id"].isna()]
print(f"매칭 안 된 리뷰 개수: {len(unmatched)}")
if len(unmatched) > 0:
    print("매칭 안 된 리뷰 샘플:")
    print(unmatched[["cafe_name", "district", "dong_name"]].head())

# 매칭된 리뷰만 사용
matched = merged[merged["cafe_id"].notna()].copy()
print(f"매칭된 리뷰 개수: {len(matched)}")

# 5) Insert SQL
insert_sql = """
INSERT INTO cafe_review
(cafe_id, cafe_name, district, dong_name, place_id, lat, lng,
 reviewer_name, rating, review_text, review_date, lang)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# NaN 값을 None으로 변환
data_list = []
for _, row in matched.iterrows():
    data_list.append([
        int(row['cafe_id']) if pd.notna(row['cafe_id']) else None,
        row['cafe_name'] if pd.notna(row['cafe_name']) else None,
        row['district'] if pd.notna(row['district']) else None,
        row['dong_name'] if pd.notna(row['dong_name']) else None,
        row.get('place_id') if pd.notna(row.get('place_id')) else None,
        float(row['lat']) if pd.notna(row.get('lat')) else None,
        float(row['lng']) if pd.notna(row.get('lng')) else None,
        row['reviewer_name'] if pd.notna(row.get('reviewer_name')) else None,
        int(row['rating']) if pd.notna(row.get('rating')) else None,
        row['review_text'] if pd.notna(row.get('review_text')) else None,
        row['review_date'] if pd.notna(row.get('review_date')) else None,
        row['lang'] if pd.notna(row.get('lang')) else None,
    ])

if data_list:
    cur.executemany(insert_sql, data_list)
    conn.commit()
    print(f"✅ 리뷰 업로드 완료: {len(data_list)}개 리뷰")
else:
    print("❌ 업로드할 리뷰가 없습니다.")


리뷰 CSV 경로: c:\Users\jiwon\Desktop\연구\space-agent\google_reviews_sample.csv
리뷰 데이터 로드 완료: 7070개 행
✅ cafe_review 테이블 생성 완료
카페 샘플 데이터 로드 완료: 2497개 카페
매칭 안 된 리뷰 개수: 0
매칭된 리뷰 개수: 7070
✅ 리뷰 업로드 완료: 7070개 리뷰


: 

In [ ]:
cur.close()
conn.close()